# Hyperparameter Search
This notebook is an implementation of ``10x_dataset_training.ipynb`` with hyperparameter search. The hyperparameter search is done using Weights and Biases (wandb) and performed with the sweep method, a grid search with random sampling. The hyperparameters are:
- ``learning_rate``: The learning rate of the optimizer.
- ``num_specific_layers``: The number of model-specific layers (i.e., self-attention layers, convolutional layers, etc.).
- ``aa_embedding_dim``: The dimension of the amino acid embedding.
- ``depth_final_dense``: The number of linear layers in the network.
- ``model_name``: The model to use. Either ``bilstm``, ``self_attention``, ``cnn``, or ``bigru``. See the ``README.md`` for more details about the implementations of these architectures. 

In [1]:
import pandas as pd
import tcellmatch.api as tm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pytorch_model_summary import summary
from torchmetrics import Accuracy
import torch
import os
import numpy as np
import wandb

# Build Model

In [2]:
USE_BIND_COUNTS = True
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
def load_model():
    ffn = tm.models.EstimatorFfn()
    indir = '../tutorial_data/'
    data = np.load(f"{indir}ffn_data_continuous_15k.npz")
    ffn.x_train = data["x_train"]
    ffn.covariates_train = data["covariates_train"]
    ffn.y_train = data["y_train"]
    ffn.x_test = data["x_test"]
    ffn.covariates_test = data["covariates_test"]
    ffn.y_test = data["y_test"]
    ffn.clone_train = data["clone_train"]
    ffn.load_idx(f'{indir}SAVED_IDX')
    
    sums_across_last_dim = np.sum(ffn.x_train, axis=-1)

    # Find rows which are not "zero-hot"
    non_zero_hot_rows = np.any(sums_across_last_dim > 0, axis=-1)
    non_zero_hot_rows = np.squeeze(non_zero_hot_rows)
    
    ffn.x_train = ffn.x_train[non_zero_hot_rows]
    ffn.y_train = ffn.y_train[non_zero_hot_rows]
    return ffn

# Add WandB Search

In [4]:
sweep_config = {
   'method': 'grid',  # can be random, grid, bayes
   'parameters': {
       'lr': {  # learning rate
           'values': [0.001, 0.01, 0.1]
       },
       'aa_embedding_dim': {
           'values': [0, 10, 26]
       },
       'depth_final_dense': {
           'values': [1, 2, 3, 5, 9]
       },
       # 'model_name': {
           # 'values': ['self-attention', 'bilstm', 'bigru', 'cnn']
          # 'values': ['self-attention']
       # },
      'attn_size': {
           # 'values': ['self-attention', 'bilstm', 'bigru', 'cnn']
          'values': [4, 8, 16, 64, 128]
       },
      'heads': {
           # 'values': ['self-attention', 'bilstm', 'bigru', 'cnn']
          'values': [4, 8, 16, 64]
       },
       # i.e., bilstm depth, SA depth, conv depth
       'specific_layer_depth': {
            'values': [1, 3, 5, 10]
       }
    },
    'metric': {
        "name": "Validation Loss",
        "goal": "minimize"
    }
}

In [5]:
def train():
    # Initialize wandb with a sample project name
    wandb.init(project="TCR fitting")

    config = wandb.config
   
    # replace your hard-coded hyperparameters with config values
    model_name = 'self-attention'
    lr = config.lr
    aa_embedding_dim = config.aa_embedding_dim
    depth_final_dense = config.depth_final_dense
    ffn = load_model()

    if model_name.lower() == 'self-attention':
        attention_size = [config.attn_size] * config.specific_layer_depth
        attention_heads = [config.heads] * config.specific_layer_depth

        ffn.build_self_attention(
            residual_connection=True,
            aa_embedding_dim=aa_embedding_dim,
            attention_size=attention_size,
            use_covariates=False,
            attention_heads=attention_heads,
            depth_final_dense=depth_final_dense,
            optimizer='adam',
            lr=lr,
            loss='pois' if USE_BIND_COUNTS else 'wbce',
            label_smoothing=0
        )
    elif model_name.lower() == 'bilstm':
        topology = [32] * config.specific_layer_depth
        ffn.build_bilstm(
            topology=topology,
            residual_connection=True,
            aa_embedding_dim=aa_embedding_dim,
            optimizer='adam',
            lr=lr,
            loss='pois' if USE_BIND_COUNTS else 'wcbe',
            label_smoothing=0,
            depth_final_dense=depth_final_dense,
            use_covariates=False,
            one_hot_y=not USE_BIND_COUNTS
        )
    elif model_name.lower() == 'bigru':
        topology= [10] * config.specific_layer_depth
        ffn.build_bigru(
            aa_embedding_dim=aa_embedding_dim,
            residual_connection=True,
            lr=lr,
            loss='pois' if USE_BIND_COUNTS else 'wbce',
        )
    elif model_name.lower() == 'cnn':
        n_conv_layers = config.specific_layer_depth
        # filter_widths = [3, 5, 3] 
        # filters = [16, 32, 64]
        pool_sizes = [2] * n_conv_layers
        pool_strides = [2] * n_conv_layers
        ffn.build_conv(
            n_conv_layers=n_conv_layers,
            depth_final_dense=depth_final_dense,
            # filter_widths=filter_widths,
            # filters=filters,
            pool_sizes=pool_sizes,
            pool_strides=pool_strides,
            loss='pois' if USE_BIND_COUNTS else 'wbce',
        )

    # Training model
    EPOCHS = 5
    batch_size = 16
    ffn.model = ffn.model.to(device=device)
    train_curve, val_curve, antigen_loss, antigen_loss_val = ffn.train(
        epochs=EPOCHS,
        batch_size=batch_size,
        log_dir='training_runs',
        allow_early_stopping=False,
        print_loss=False,
        lr_schedule_factor=0.99999,
        use_wandb=True
    )
   
    # Log metrics with wandb
    # wandb.log({'Train Loss': train_curve[-1], 'Validation Loss': val_curve[-1]})
    wandb.log({"Validation Loss": val_curve[-1], "_wandb": {"desc": "Validation Loss"}})

In [6]:
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: ydbdep0d
Sweep URL: https://wandb.ai/jmboesen/uncategorized/sweeps/ydbdep0d


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: ubjg6ub5 with config:
wandb: 	aa_embedding_dim: 0
wandb: 	attn_size: 4
wandb: 	depth_final_dense: 1
wandb: 	heads: 4
wandb: 	lr: 0.001
wandb: 	specific_layer_depth: 1
wandb: Currently logged in as: jmboesen. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


started training...
Number of observations in evaluation data: 1516
Number of observations in training data: 13484


Validation Loss,▁
antigens/antigen 0,█▄▃▂▁
antigens/antigen 0 val,██▁▄▇
antigens/antigen 1,█▅▃▂▁
antigens/antigen 1 val,▂█▆▁█
antigens/antigen 10,█▄▂▂▁
antigens/antigen 10 val,▁▃▅▆█
antigens/antigen 11,█▅▃▂▁
antigens/antigen 11 val,▂▁▂█▃
antigens/antigen 12,█▃▂▂▁
antigens/antigen 12 val,▂▁▇▁█


wandb: Agent Starting Run: 27dk9u4d with config:
wandb: 	aa_embedding_dim: 0
wandb: 	attn_size: 4
wandb: 	depth_final_dense: 1
wandb: 	heads: 4
wandb: 	lr: 0.001
wandb: 	specific_layer_depth: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


started training...
Number of observations in evaluation data: 1480
Number of observations in training data: 13520


...

wandb: Agent Starting Run: q9dnulr5 with config:
wandb: 	aa_embedding_dim: 0
wandb: 	attn_size: 4
wandb: 	depth_final_dense: 2
wandb: 	heads: 8
wandb: 	lr: 0.001
wandb: 	specific_layer_depth: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


started training...
Number of observations in evaluation data: 1470
Number of observations in training data: 13530


Validation Loss,▁
antigens/antigen 0,█▃▄▁▁
antigens/antigen 0 val,█▂▁▂▂
antigens/antigen 1,█▂▃▁▂
antigens/antigen 1 val,▃▁▆█▇
antigens/antigen 10,█▂▁▁▁
antigens/antigen 10 val,██▁▅▄
antigens/antigen 11,█▁▃▁▁
antigens/antigen 11 val,█▆▁▁▂
antigens/antigen 12,█▅▄▃▁
antigens/antigen 12 val,▄▅▁▇█


wandb: Agent Starting Run: guxsz928 with config:
wandb: 	aa_embedding_dim: 0
wandb: 	attn_size: 4
wandb: 	depth_final_dense: 2
wandb: 	heads: 8
wandb: 	lr: 0.001
wandb: 	specific_layer_depth: 5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


started training...
Number of observations in evaluation data: 1511
Number of observations in training data: 13489
